In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pickle
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
pip install sentence_transformers

https://arxiv.org/abs/1908.10084

In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
import torch 
from torch import nn
cos_sim = nn.CosineSimilarity(dim=0)

In [6]:
def cal(s1,s2):
    s1_enbeding = model.encode( s1 )
    s2_enbeding = model.encode( s2 )
    a1 = list(zip(s1,s1_enbeding))
    a2 = list(zip(s2,s2_enbeding))
    all_list = []
    for i in a1:
        max_j = []
        for j in a2:
            sim = cos_sim(torch.tensor(i[1]),torch.tensor(j[1]))
            max_j.append([i[0],j[0],sim])
        all_list.append(max(max_j,key=lambda x: x[-1]))
    for i in range(len(all_list)):
        all_list[i][-1] = all_list[i][-1].numpy().tolist()
#     all_list.sort(key=lambda x:x[-1],reverse=True)
    return all_list

In [7]:
ar  = cal(['access the download music'],['download music'])
ar

In [5]:
data = pickle.load(open('../input/fiters-pk/pk2/pk2/{}.pk'.format('YouTube_privacy_reviews'),'rb'))
s1 = data['s1']
s2 = data['s2']
print(s2)
fiter = ['have the video','shoot','watch']
s2 = [i.replace('watch engage','engage') for i in s2 if i not in fiter]
s2 = [i.replace('the youtube ','').replace('youtube ','') for i in s2]
arr = cal(s1,s2)
with open('{}-phrase-bert.csv'.format('YouTube_privacy_reviews'),'w') as f:
    for (i,j,z),id in zip(arr,data['id']):
        z = str(z)
        f.write('"{}","{}",{},{}'.format(i,j,z,id)+'\n')


In [ ]:
# ../input/fiters-pk/pk/pk/Facebook_privacy_reviews.pk
# ../input/fiters-pk/pk2/pk2

In [8]:
a = ['Facebook_privacy_reviews','Instagram_privacy_reviews','Spotify_privacy_reviews','TikTok_privacy_reviews','Twitter_privacy_reviews',
    'YouTube_privacy_reviews','pocketguard']
for i in a:
    data = pickle.load(open('../input/fiters-pk/pk2/pk2/{}.pk'.format(i),'rb'))
    s1 = data['s1']
    s2 = data['s2']
    arr = cal(s1,s2)
    with open('{}-phrase-bert.csv'.format(i),'w') as f:
        for (i,j,z),id in zip(arr,data['id']):
            z = str(z)
            f.write('"{}","{}",{},{}'.format(i,j,z,id)+'\n')